## Imports

In [0]:
from pyspark.sql.functions import col
from pyspark.sql import Window
import pyspark.sql.functions as F
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator


import random

import mlflow
print(mlflow.__version__)

import os
os.environ['PYSPARK_PIN_THREAD'] = 'false'
spark.conf.set("spark.databricks.mlflow.trackMLlib.enabled", "true")

RANDOM_SEED = 0
# Define experiment name with proper Databricks path
EXPERIMENT_NAME = "/Shared/team_2_2/mlflow-baseline"
# Create the experiment if it doesn't exist
try:
    experiment = mlflow.get_experiment_by_name(EXPERIMENT_NAME)
    if experiment is None:
        experiment_id = mlflow.create_experiment(EXPERIMENT_NAME)
        print(f"Created new experiment with ID: {experiment_id}")
    else:
        print(f"Using existing experiment: {experiment.name}")
    mlflow.set_experiment(EXPERIMENT_NAME)
except Exception as e:
    print(f"Error with experiment setup: {e}")
    # Fallback to default experiment in workspace
    mlflow.set_experiment(f"/Users/{dbutils.notebook.entry_point.getDbutils().notebook().getContext().userName().get()}/default")



## Helper Functions


In [0]:
def checkpoint_dataset(dataset, file_path):
    # Create base folder
    section = "2"
    number = "2"
    base_folder = f"dbfs:/student-groups/Group_{section}_{number}"
    dbutils.fs.mkdirs(base_folder)
    # Create subfolders if file_path contains directories
    full_path = f"{base_folder}/{file_path}.parquet"
    subfolder = "/".join(full_path.split("/")[:-1])
    dbutils.fs.mkdirs(subfolder)
    # Save dataset as a parquet file
    dataset.write.mode("overwrite").parquet(full_path)
    print(f"Checkpointed {file_path}")

In [0]:

def checkpoint_dataset(dataset, file_path):
    # Create folder
    section = "2"
    number = "2"
    folder_path = f"dbfs:/student-groups/Group_{section}_{number}"
    dbutils.fs.mkdirs(folder_path)
    # Save df_weather as a parquet file
    dataset.write.parquet(f"{folder_path}/{file_path}.parquet")
    print(f"Checkpointed {file_path}")

## Datasets

In [0]:
# Airline Data    
df_flights = spark.read.parquet(f"dbfs:/mnt/mids-w261/datasets_final_project_2022/parquet_airlines_data_3m/")
print(df_flights.count())
display(df_flights)

## Column analysis - Can remove later
- OP_UNIQUE_CARRIER == OP_CARRIER_AIRLINE_ID == OP_CARRIER == OP_CARRIER_FL_NUM
- TAIL_NUMBER --> License Plate Number
- ORIGIN_AIRPORT_SEQ_ID == ORIGIN_CITY_MARKET_ID == ORIGIN == ORIGIN_CITY_NAME == ORIGIN_STATE_ABR == ORIGIN_STATE_FIPS (code for state) == ORIGIN_STATE_NM == ORIGIN_WAC (origin airport, world area code)
- DEST_AIRPORT_ID == DEST_AIRPORT_SEQ_ID == DEST_CITY_MARKET_ID == DEST == DEST_CITY_NAME == DEST_STATE_ABR == DEST_STATE_FIPS == DEST_STATE_NAME == DEST_WAC
- CRS_DEP_TIME -> Scheduled departure time in the computer reservation system
---
- DEP_TIME -> Actual Dept time
- DEPT_DELAY -> difference in minutes between scheduled and actual departure time
- DEPT_DELAY_NEW -> early flights are 0
- DEP_DEL15
- DEP_DELAY_GROUP
- DEP_TIME_BLK
- TAXI_OUT -> Taxi out time in minutes
- WHEELS OFF
- WHEELS ON - Time at landing (local time)
- TAXI_IN
- CRS ARR TIME - scheduled arrival time
- ARR_TIME - actual arrival time
- ARR_DELAY
- ARR_DELAY_NEW
- ARR_DEL15
- ARR_DELAY_GROUP
- ARR_TIME_BLK
- CANCELLED, CANELLATION_CODE, DIVERTED
- CRS_ELAPSED_TIME - scheduled flight time
- ACTUAL_ELAPSED_TIME
- AIR_TIME = flight time in minutes
- FLIGHTS = Number of flights (Idk what this means)
- DISTANCE
- DISTANCE_GROUP (every 250 miles)
- CARRIER_DELAY
- WEATHER_DELAY
- NAS_DELAY (National air system delay)
- SECURITY_DELAY
- LATE_AIRCRAFT_DELAY
- FIRST_DEP_TIME - first gate departure time at origin airport (nulls see what to do with them)
- TOTAL_ADD_GTIME - total ground time away from gate for gate return or cancelled flight
- LONGEST_ADD_GTME - longest time away from gate for gate return or cancelled flight
- A BUNCH OF DIVERTED AIRPORT COLUMNS (do some eda, they seem empty)

# Things to keep in mind
- Predict two hours before
- Remove all the delay columns
- Are we only predicting departure delays or arrival delays also? For example, the pilot misses the landing, and has to circle back for 20 minutes. Should we solve for that? I don't think we should.

## Preprocessing / Cleanup

In [0]:
# Make a copy of the dataset
df_flights = df_flights.cache()

In [0]:
# combine date and scheduled departure time

df_flights = df_flights.withColumn(
    "utc_timestamp",
    F.to_timestamp(
        F.concat(
            F.col("FL_DATE"),
            F.lit(" "),
            F.lpad(F.col("CRS_DEP_TIME").cast("string"), 4, "0")
        ),
        "yyyy-MM-dd HHmm"
    )
)

In [0]:
# impute null values for tail numbers
#   - all replaced tail numbers will start with 'X' followed by a randomized 5 digit number

df_flights = df_flights.withColumn(
    "TAIL_NUM",
    F.when(
        F.col("TAIL_NUM").isNull(),
        F.concat(
            F.lit("X"),
            (F.floor(F.rand() * 89999) + 10000).cast("string")
        )
    ).otherwise(F.col("TAIL_NUM"))
)

## Model Iterations

In [0]:
checkpoint_path = f"dbfs:/student-groups/Group_2_2"
dataset_path = f"{checkpoint_path}/3_month/raw_data/training_splits"

# Read datasets from checkpoint
train_df = spark.read.parquet(f"{dataset_path}/train.parquet")
validation_df = spark.read.parquet(f"{dataset_path}/validation.parquet")

In [0]:
# Feature Engineering

## CRS_DEP_TIME is local time so we can use this feature 
## But in order to use it, we have to convert it to minutes since midnight
## Otherwise the timing will be off b/c it's not true UTC

train_df = train_df.\
        withColumn("CRS_DEP_MINUTES", (F.floor(F.col("CRS_DEP_TIME") / 100) * 60 + (F.col("CRS_DEP_TIME") % 100))).\
        drop("CRS_DEP_TIME").\
        drop("CRS_ARR_TIME")

validation_df = validation_df.\
        withColumn("CRS_DEP_MINUTES", (F.floor(F.col("CRS_DEP_TIME") / 100) * 60 + (F.col("CRS_DEP_TIME") % 100))).\
        drop("CRS_DEP_TIME").\
        drop("CRS_ARR_TIME")

### Feature Eng.

#### Was the previous flight delayed? And by how much was the previous flight delayed?

In [0]:
train_df = train_df.cache()

In [0]:
train_df = train_df \
    .withColumn("prev_flight_delay_in_minutes", F.lag("DEP_DELAY_NEW", 1) \
        .over(Window.partitionBy("TAIL_NUM") \
        .orderBy("utc_timestamp"))) \
    .withColumn("prev_flight_delay_in_minutes", F.when(F.col("prev_flight_delay_in_minutes").isNull(), -1) \
        .otherwise(F.col("prev_flight_delay_in_minutes"))) \
    .withColumn("prev_flight_delay", F.when(F.col("prev_flight_delay_in_minutes") > 15, 1) \
        .otherwise(F.lit(0)))
    
validation_df = validation_df \
    .withColumn("prev_flight_delay_in_minutes", F.lag("DEP_DELAY_NEW", 1) \
        .over(Window.partitionBy("TAIL_NUM") \
        .orderBy("utc_timestamp"))) \
    .withColumn("prev_flight_delay_in_minutes", F.when(F.col("prev_flight_delay_in_minutes").isNull(), -1) \
        .otherwise(F.col("prev_flight_delay_in_minutes"))) \
    .withColumn("prev_flight_delay", F.when(F.col("prev_flight_delay_in_minutes") > 15, 1) \
        .otherwise(F.lit(0)))


### Number of delays before in the last 4 hours

In [0]:
useful_columns = [
    "QUARTER",
    "MONTH",
    "YEAR",
    "DAY_OF_MONTH",
    "DAY_OF_WEEK",
    "OP_CARRIER",
    "TAIL_NUM",
    "ORIGIN_AIRPORT_SEQ_ID",
    "DEST_AIRPORT_SEQ_ID",
    "CRS_DEP_MINUTES",
    "CRS_ELAPSED_TIME",
    "DISTANCE",
    "DEP_DELAY_NEW",
    "utc_timestamp",
    "prev_flight_delay_in_minutes",
    "prev_flight_delay",
    "origin_delays_4h"
]

In [0]:
window_4h = Window \
    .partitionBy("ORIGIN_AIRPORT_SEQ_ID") \
    .orderBy(F.col("utc_timestamp").cast("long")) \
    .rangeBetween(-14400, -7200) # 4 hours to 2 hours before

train_df = train_df \
    .withColumn("origin_delays_4h", F.count(F.when(F.col("DEP_DELAY_NEW") > 15, 1)) \
        .over(window_4h)
    )
validation_df = validation_df \
    .withColumn("origin_delays_4h", F.count(F.when(F.col("DEP_DELAY_NEW") > 15, 1)) \
        .over(window_4h)
    )

In [0]:
baselines_columns = [
    "QUARTER",
    "MONTH",
    "YEAR",
    "DAY_OF_MONTH",
    "DAY_OF_WEEK",
    "OP_CARRIER",
    "TAIL_NUM",
    "ORIGIN_AIRPORT_SEQ_ID",
    "DEST_AIRPORT_SEQ_ID",
    "CRS_DEP_MINUTES",
    "CRS_ELAPSED_TIME",
    "DISTANCE",
    "DEP_DELAY_NEW",
    # "utc_timestamp",
    "prev_flight_delay_in_minutes",
    "prev_flight_delay",
    "origin_delays_4h"
]


train_df = train_df.filter(F.col("DEP_DELAY_NEW").isNotNull()).select(baselines_columns)
validation_df = validation_df.filter(F.col("DEP_DELAY_NEW").isNotNull()).select(baselines_columns)

In [0]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.regression import RandomForestRegressor

# Categorical encoding
carrier_indexer = StringIndexer(inputCol="OP_CARRIER", outputCol="carrier_idx", handleInvalid="keep")
origin_indexer = StringIndexer(inputCol="ORIGIN_AIRPORT_SEQ_ID", outputCol="origin_idx", handleInvalid="keep")
dest_indexer = StringIndexer(inputCol="DEST_AIRPORT_SEQ_ID", outputCol="dest_idx", handleInvalid="keep")
tail_num_indexer = StringIndexer(inputCol="TAIL_NUM", outputCol="tail_num_idx", handleInvalid="keep")

carrier_encoder = OneHotEncoder(inputCol="carrier_idx", outputCol="carrier_vec")
origin_encoder = OneHotEncoder(inputCol="origin_idx", outputCol="origin_vec")
dest_encoder = OneHotEncoder(inputCol="dest_idx", outputCol="dest_vec")
tail_num_encoder = OneHotEncoder(inputCol="tail_num_idx", outputCol="tail_num_vec")



In [0]:
# Assemble all features
assembler = VectorAssembler(
    inputCols=[
        "QUARTER",
        "MONTH", 
        "YEAR",
        "DAY_OF_MONTH",
        "DAY_OF_WEEK",
        "carrier_vec",
        "origin_vec",
        "dest_vec",
        "tail_num_vec",
        "CRS_DEP_MINUTES",
        "CRS_ELAPSED_TIME",
        "DISTANCE",
        "prev_flight_delay_in_minutes",
        "prev_flight_delay",
        "origin_delays_4h"
    ],
    outputCol="features"
)

In [0]:
# linear regression baseline
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
from mlflow.models import infer_signature

mlflow.spark.autolog()
with mlflow.start_run(run_name="RF - origin airport delays"):
    MODEL_NAME = "RF_ORIGIN_DELAYS"

    linear_reg = LinearRegression(
        featuresCol="features",
        labelCol="DEP_DELAY_NEW",
        # Linear Regression has different parameters than Random Forest
        maxIter=10, 
        regParam=0.3
    )
    # rf = RandomForestRegressor(
    #     featuresCol="features",  
    #     labelCol="DEP_DELAY_NEW",   
    #     numTrees=20,
    #     maxDepth=10
    # )

    # Create pipeline
    pipeline = Pipeline(stages=[
        carrier_indexer, origin_indexer, dest_indexer, tail_num_indexer,
        carrier_encoder, origin_encoder, dest_encoder, tail_num_encoder,
        assembler,
        linear_reg
        # rf
    ])

    model = pipeline.fit(train_df)
    training_predictions = model.transform(train_df)
    validation_predictions = model.transform(validation_df)

    mae_evaluator = RegressionEvaluator(
        labelCol="DEP_DELAY_NEW",      
        predictionCol="prediction", 
        metricName="mae"           
    )

    rmse_evaluator = RegressionEvaluator(
        labelCol="DEP_DELAY_NEW",      
        predictionCol="prediction", 
        metricName="rmse"
    )

    # Calculate MAE
    mae_t = mae_evaluator.evaluate(training_predictions)
    mae_v = mae_evaluator.evaluate(validation_predictions)
    # Calculate RMSE
    rmse_t = rmse_evaluator.evaluate(training_predictions)
    rmse_v = rmse_evaluator.evaluate(validation_predictions)

    signature = infer_signature(train_df, training_predictions)

    mlflow.spark.log_model(
        model, 
        MODEL_NAME,
        input_example=train_df.limit(1).toPandas(),
        signature=signature,
        registered_model_name="flight_delay_prediction_baseline"
        )

    mlflow.log_metric("train_mae", mae_t)
    mlflow.log_metric("validation_mae", mae_v)
    mlflow.log_metric("train_rmse", rmse_t)
    mlflow.log_metric("validation_rmse", rmse_v)


## Checkpoint results to MLflow?

## Develop and Run Pipeline

## Cross Validation

## Save metrics, pipeline, any other steps

## Feature Engineering Ideas
- Previous flight delay in minutes for the aircraft [DONE] - this added value to the linear regression model!
- Number of delayed flights from 4 hours (DONE)
- Number of delays in the route in the last 30 days
- Time between landing and scheduled current flight
- Airport + utc time type of delay - Ohare at 6PM is always late
- Number of delayed flights in departure and arrival location (total or 4 hours before, 6 hours before, etc.)
- Average delay time by airport
- Average taxi out time by airport/flight